# Enterprise Work Order Batch Processing Pipeline

This notebook demonstrates an **automated batch processing pipeline** that processes high-value work orders using:
- **Snowflake Streams** to capture work order changes
- **Snowflake Tasks** for automated batch processing weekly
- **Cortex AI** for intelligent summarization

## 🎯 Business Focus
Only work orders **≥ $25,000** trigger AI summarization for executive attention.

**✅ TESTED AND WORKING** - This pipeline has been validated end-to-end.

## Prerequisites

Ensure your role has the necessary privileges:

```sql
GRANT EXECUTE TASK ON ACCOUNT TO ROLE <YOUR_ROLE>;
```

In [ ]:
-- Set up the environment
USE ROLE ACCOUNTADMIN;
USE DATABASE DEMODB;
USE SCHEMA WORK_ORDER_MANAGEMENT;

-- 🎯 BUSINESS CONFIGURATION
-- Pipeline processes work orders >= $25,000
-- Threshold is hardcoded in stored procedure for reliability

SELECT 
    'PIPELINE CONFIGURATION' as STATUS,
    'High-value threshold: $25,000' as COST_THRESHOLD,
    'Batch processing: Weekly on Mondays at 9 AM EST' as SCHEDULE,
    'Active statuses: Open, In Progress, Pending Approval' as STATUSES;

## Step 1: Create Stream

Create a Snowflake Stream to capture changes to the `ENTERPRISE_WORK_ORDERS` table. The stream provides an audit trail of all changes, while business filtering for batch processing happens in the stored procedure.

In [ ]:
-- Create stream to monitor work orders table
CREATE OR REPLACE STREAM ENTERPRISE_WORK_ORDERS_STREAM
ON TABLE ENTERPRISE_WORK_ORDERS
COMMENT = 'Stream to capture new work orders for batch processing';

-- Verify stream creation
SELECT 
    'Stream created successfully' AS STATUS,
    SYSTEM$STREAM_HAS_DATA('ENTERPRISE_WORK_ORDERS_STREAM') AS HAS_DATA,
    CURRENT_TIMESTAMP() AS CREATED_AT;

## Step 2: Create Processing Stored Procedure

**Key Business Logic:**
- Only processes NEW work orders (INSERT operations)
- Cost threshold: **≥ $25,000** (hardcoded for task reliability)
- Status filter: Active work orders only
- Duplicate prevention: Won't reprocess existing summaries

**✅ TESTED AND WORKING** - This is the validated version that successfully processes high-value work orders in batches.

In [ ]:
CREATE OR REPLACE PROCEDURE PROCESS_HIGH_COST_WORK_ORDERS()
RETURNS STRING
LANGUAGE SQL
EXECUTE AS CALLER
AS
$$
DECLARE
    min_cost_threshold NUMBER DEFAULT 25000;  -- Hardcoded for task reliability
    result_message STRING;
BEGIN
    -- Process new high-cost work orders from stream in batch
    INSERT INTO WORK_ORDER_SUMMARIES (WORK_ORDER_ID, SUMMARY, CREATED_TIMESTAMP)
    WITH new_high_cost_orders AS (
        -- 🎯 BUSINESS LOGIC APPLIED HERE:
        -- 1. Only INSERT operations (new work orders)
        -- 2. Cost >= $25,000 (hardcoded threshold)
        -- 3. Active status only (Open, In Progress, Pending Approval)
        -- 4. No existing summary (avoid duplicates)
        SELECT 
            s.WORK_ORDER_ID,
            s.WORK_ORDER_NOTES
        FROM ENTERPRISE_WORK_ORDERS_STREAM s
        WHERE s.METADATA$ACTION = 'INSERT'  -- Only new inserts
          AND s.TOTAL_COST >= :min_cost_threshold  -- HIGH-VALUE THRESHOLD
          AND s.STATUS IN ('Open', 'In Progress', 'Pending Approval')  -- ACTIVE STATUSES ONLY
          -- Ensure we don't already have a summary
          AND NOT EXISTS (
              SELECT 1 FROM WORK_ORDER_SUMMARIES ws 
              WHERE ws.WORK_ORDER_ID = s.WORK_ORDER_ID
          )
    ),
    ai_summaries AS (
        -- Generate AI summaries for qualifying work orders
        SELECT 
            n.WORK_ORDER_ID,
            SNOWFLAKE.CORTEX.COMPLETE(
                'claude-3-5-sonnet',
                ARRAY_CONSTRUCT(
                    OBJECT_CONSTRUCT('role', 'user', 'content', 'Summarize these technician notes: ' || n.WORK_ORDER_NOTES)
                ),
                OBJECT_CONSTRUCT('max_tokens', 256)
            ):choices[0]:messages::STRING AS ai_summary
        FROM new_high_cost_orders n
    )
    SELECT 
        s.WORK_ORDER_ID,
        s.ai_summary,
        CURRENT_TIMESTAMP()
    FROM ai_summaries s
    WHERE s.ai_summary IS NOT NULL;

    -- Create result message
    result_message := 'Processed high-cost work orders from stream. Cost threshold: $' || min_cost_threshold;

    RETURN result_message;

END;
$$;

SELECT 'Stored procedure created successfully' AS STATUS;

## 📋 Stored Procedure Logic Explained

The `PROCESS_HIGH_COST_WORK_ORDERS()` procedure implements a **three-stage filtering and processing pattern** that you can adapt for similar use cases:

### **Stage 1: Business Rule Filtering (`new_high_cost_orders` CTE)**
```sql
-- Applies 4 key business filters:
WHERE s.METADATA$ACTION = 'INSERT'           -- Only new records
  AND s.TOTAL_COST >= :min_cost_threshold    -- Cost threshold ($25K)
  AND s.STATUS IN ('Open', 'In Progress', 'Pending Approval')  -- Active only
  AND NOT EXISTS (SELECT 1 FROM WORK_ORDER_SUMMARIES...)       -- No duplicates
```

**💡 Implementation Pattern**: Use this approach when you need to:
- Filter stream data by business criteria
- Prevent duplicate processing
- Apply multiple conditions efficiently

### **Stage 2: AI Processing (`ai_summaries` CTE)**
```sql
-- Generates AI summaries using Snowflake Cortex
SNOWFLAKE.CORTEX.COMPLETE(
    'claude-3-5-sonnet',                    -- Model selection
    ARRAY_CONSTRUCT(OBJECT_CONSTRUCT(...)), -- Conversation format
    OBJECT_CONSTRUCT('max_tokens', 256)     -- Response limits
):choices[0]:messages::STRING               -- Extract clean text
```

**💡 Implementation Pattern**: Use this approach when you need to:
- Process data through AI models in batch
- Handle AI response formatting
- Control AI output parameters

### **Stage 3: Data Persistence**
```sql
-- Inserts results with timestamp
INSERT INTO WORK_ORDER_SUMMARIES (WORK_ORDER_ID, SUMMARY, CREATED_TIMESTAMP)
SELECT s.WORK_ORDER_ID, s.ai_summary, CURRENT_TIMESTAMP()
FROM ai_summaries s
WHERE s.ai_summary IS NOT NULL;  -- Only successful AI responses
```

**💡 Implementation Pattern**: Use this approach when you need to:
- Store processed results with audit trails
- Handle potential AI processing failures gracefully
- Maintain data quality (NULL filtering)

### **�� Key Design Decisions**

1. **Hardcoded Threshold**: `25000` instead of session variables for task reliability
2. **Stream Metadata**: Uses `METADATA$ACTION = 'INSERT'` to process only new records
3. **CTE Pattern**: Separates filtering, processing, and persistence for clarity
4. **Error Handling**: `WHERE s.ai_summary IS NOT NULL` prevents storing failed AI calls
5. **Idempotent Design**: `NOT EXISTS` check prevents duplicate processing

**✅ This pattern can be adapted for any stream-based AI processing pipeline!**

## Step 3: Create Automated Task

**Weekly Batch Processing**: Task runs every Monday at 9 AM EST but only when the stream has new data, processing work orders in batches to save compute costs.

In [ ]:
CREATE OR REPLACE TASK PROCESS_HIGH_COST_WORK_ORDERS_TASK
    WAREHOUSE = COMPUTE_WH
    SCHEDULE = 'USING CRON 0 9 * * MON America/New_York'
    COMMENT = 'Automatically process high-cost work orders (>=$25K) for AI summarization - runs weekly on Mondays at 9 AM EST'
    WHEN (
        -- Only execute when stream has data
        -- The stored procedure handles filtering for high-value work orders
        SYSTEM$STREAM_HAS_DATA('ENTERPRISE_WORK_ORDERS_STREAM')
    )
AS
    CALL PROCESS_HIGH_COST_WORK_ORDERS();

-- Resume the task to start processing
ALTER TASK PROCESS_HIGH_COST_WORK_ORDERS_TASK RESUME;

SELECT 'Task created and resumed successfully' AS STATUS;

## Step 4: Clean Setup for Testing

**Important**: Remove any existing test data first to ensure clean demonstration of the batch processing pipeline.

In [ ]:
-- Clean up any existing test data to ensure clean demo
DELETE FROM WORK_ORDER_SUMMARIES WHERE WORK_ORDER_ID LIKE 'DEMO-%';
DELETE FROM ENTERPRISE_WORK_ORDERS WHERE WORK_ORDER_ID LIKE 'DEMO-%';

-- Verify cleanup
SELECT 
    'Cleanup completed' AS STATUS,
    (SELECT COUNT(*) FROM ENTERPRISE_WORK_ORDERS WHERE WORK_ORDER_ID LIKE 'DEMO-%') AS REMAINING_WORK_ORDERS,
    (SELECT COUNT(*) FROM WORK_ORDER_SUMMARIES WHERE WORK_ORDER_ID LIKE 'DEMO-%') AS REMAINING_SUMMARIES;

-- Show stream status after cleanup
SELECT 
    'Stream Status After Cleanup' AS CHECK_TYPE,
    SYSTEM$STREAM_HAS_DATA('ENTERPRISE_WORK_ORDERS_STREAM') AS HAS_DATA,
    (SELECT COUNT(*) FROM ENTERPRISE_WORK_ORDERS_STREAM) AS STREAM_RECORD_COUNT;

## Step 5: Insert Test Data

Now insert fresh test work orders to validate that only high-cost ones (≥$25K) get processed in the next batch run.

In [ ]:
-- Insert HIGH-COST test work order (should trigger processing)
INSERT INTO ENTERPRISE_WORK_ORDERS (
    WORK_ORDER_ID, EQUIPMENT_TYPE, FACILITY, STATUS, PRIORITY, URGENCY,
    TOTAL_COST, LABOR_COST, PARTS_COST, LABOR_HOURS,
    CREATED_DATE, SCHEDULED_DATE, WORK_ORDER_NOTES,
    FAILURE_MODE, ESTIMATED_DOWNTIME_HOURS, BUSINESS_IMPACT_SCORE,
    SAFETY_RISK_LEVEL, COMPLIANCE_REQUIRED, SPECIALIZED_SKILLS_REQUIRED, CONTRACTOR_COST
) VALUES (
    'DEMO-HIGH-001',
    'Critical Production Line',
    'Manufacturing Plant A',
    'Open',
    'Critical',
    'High',
    50000.00,  -- HIGH COST: Above $25K threshold ✅
    20000.00,
    25000.00,
    80.0,
    CURRENT_DATE(),
    CURRENT_DATE() + 3,
    'CRITICAL: Main production line failure requiring immediate repair. High-cost emergency parts needed.',
    'Mechanical Failure',
    24.0,
    9,
    'High',
    TRUE,
    TRUE,
    5000.00
);

-- Insert LOW-COST test work order (should NOT trigger processing)
INSERT INTO ENTERPRISE_WORK_ORDERS (
    WORK_ORDER_ID, EQUIPMENT_TYPE, FACILITY, STATUS, PRIORITY, URGENCY,
    TOTAL_COST, LABOR_COST, PARTS_COST, LABOR_HOURS,
    CREATED_DATE, SCHEDULED_DATE, WORK_ORDER_NOTES,
    FAILURE_MODE, ESTIMATED_DOWNTIME_HOURS, BUSINESS_IMPACT_SCORE,
    SAFETY_RISK_LEVEL, COMPLIANCE_REQUIRED, SPECIALIZED_SKILLS_REQUIRED, CONTRACTOR_COST
) VALUES (
    'DEMO-LOW-001',
    'Office Equipment',
    'Admin Building',
    'Open',
    'Low',
    'Low',
    2000.00,  -- LOW COST: Below $25K threshold ❌
    1000.00,
    800.00,
    4.0,
    CURRENT_DATE(),
    CURRENT_DATE() + 7,
    'Routine maintenance on office printers and workstations.',
    'Routine Maintenance',
    1.0,
    2,
    'Low',
    FALSE,
    FALSE,
    200.00
);

SELECT 'Fresh test data inserted' AS STATUS;

-- Verify stream captured the new inserts
SELECT 
    'Stream Data Check' AS CHECK_TYPE,
    COUNT(*) AS NEW_RECORDS_IN_STREAM,
    COUNT(CASE WHEN TOTAL_COST >= 25000 THEN 1 END) AS HIGH_VALUE_IN_STREAM,
    COUNT(CASE WHEN TOTAL_COST < 25000 THEN 1 END) AS LOW_VALUE_IN_STREAM
FROM ENTERPRISE_WORK_ORDERS_STREAM
WHERE METADATA$ACTION = 'INSERT' AND WORK_ORDER_ID LIKE 'DEMO-%';

## Step 6: Trigger Batch Processing

Manually trigger the task to demonstrate immediate batch processing (normally runs weekly on Mondays at 9 AM EST).

In [ ]:
-- Manually trigger the task for immediate testing
EXECUTE TASK PROCESS_HIGH_COST_WORK_ORDERS_TASK;

SELECT 'Batch processing task executed - checking results...' AS STATUS;

## Wait 2 minutes for the task to run 


## Step 7: Validate Results

Verify that the batch processing worked correctly - only high-cost work orders should have summaries.

In [ ]:
-- Validate business logic: Only high-cost work orders should have summaries
SELECT 
    '🧪 BATCH PROCESSING VALIDATION' AS TEST_TYPE,
    w.WORK_ORDER_ID,
    w.TOTAL_COST,
    CASE 
        WHEN w.TOTAL_COST >= 25000 THEN 'HIGH (should have summary)'
        ELSE 'LOW (should NOT have summary)'
    END AS EXPECTED_CATEGORY,
    CASE 
        WHEN s.WORK_ORDER_ID IS NOT NULL THEN 'HAS SUMMARY ✅' 
        ELSE 'NO SUMMARY ❌' 
    END AS ACTUAL_RESULT,
    CASE 
        WHEN w.TOTAL_COST >= 25000 AND s.WORK_ORDER_ID IS NOT NULL THEN '✅ CORRECT'
        WHEN w.TOTAL_COST < 25000 AND s.WORK_ORDER_ID IS NULL THEN '✅ CORRECT'
        ELSE '❌ ERROR'
    END AS VALIDATION_STATUS
FROM ENTERPRISE_WORK_ORDERS w
LEFT JOIN WORK_ORDER_SUMMARIES s ON w.WORK_ORDER_ID = s.WORK_ORDER_ID
WHERE w.WORK_ORDER_ID LIKE 'DEMO-%'
ORDER BY w.TOTAL_COST DESC;

In [ ]:
-- Show the generated AI summary for validation
SELECT 
    '🤖 AI GENERATED SUMMARY' AS CONTENT_TYPE,
    s.WORK_ORDER_ID,
    w.TOTAL_COST,
    s.SUMMARY,
    s.CREATED_TIMESTAMP
FROM WORK_ORDER_SUMMARIES s
JOIN ENTERPRISE_WORK_ORDERS w ON s.WORK_ORDER_ID = w.WORK_ORDER_ID
WHERE s.WORK_ORDER_ID LIKE 'DEMO-%'
ORDER BY s.CREATED_TIMESTAMP DESC;

## Step 8: Final Clean Up

Remove test data after validation to keep the environment clean.

In [ ]:
-- Clean up test data after demonstration
DELETE FROM WORK_ORDER_SUMMARIES WHERE WORK_ORDER_ID LIKE 'DEMO-%';
DELETE FROM ENTERPRISE_WORK_ORDERS WHERE WORK_ORDER_ID LIKE 'DEMO-%';

SELECT 'Test data cleaned up successfully' AS STATUS;

-- Final verification
SELECT 
    'Final Cleanup Verification' AS CHECK_TYPE,
    (SELECT COUNT(*) FROM ENTERPRISE_WORK_ORDERS WHERE WORK_ORDER_ID LIKE 'DEMO-%') AS REMAINING_WORK_ORDERS,
    (SELECT COUNT(*) FROM WORK_ORDER_SUMMARIES WHERE WORK_ORDER_ID LIKE 'DEMO-%') AS REMAINING_SUMMARIES;

## 🎯 Weekly Batch Processing Pipeline Summary

### **✅ VALIDATED BUSINESS LOGIC:**

**High-Value Work Order Criteria:**
- **Cost Threshold**: ≥ $25,000 (hardcoded for reliability)
- **Status Filter**: Active work orders only ('Open', 'In Progress', 'Pending Approval')
- **Duplicate Prevention**: Won't reprocess work orders with existing summaries

### **🏗️ Architecture Components:**

1. **📊 Stream**: Captures work order changes for audit trail
2. **🧠 Stored Procedure**: Applies business filtering and generates AI summaries
3. **⚡ Task**: Executes batch processing weekly on Mondays at 9 AM EST when new data is available
4. **📈 Testing**: Available via `test_pipeline_end_to_end.sql` script

### **🤖 AI Integration:**
- **Model**: Claude-3.5-Sonnet via Cortex Complete
- **Focus**: Concise summaries for executive attention
- **Output**: Automatically stored in `WORK_ORDER_SUMMARIES` table

### **💡 Key Benefits:**
- **Cost Efficient**: Only processes high-value work orders
- **Weekly Batch Processing**: Automated processing every Monday at 9 AM EST
- **Reliable**: Hardcoded thresholds prevent session variable issues
- **Auditable**: Complete stream history for compliance and analysis
- **Clean Testing**: Proper cleanup ensures repeatable demonstrations

**🚀 The weekly batch processing pipeline is now ready for production use!**

**For comprehensive testing**, use the `test_pipeline_end_to_end.sql` script which validates the entire pipeline with detailed business logic verification.